Dataset: https://www.kaggle.com/tawsifurrahman/covid19-radiography-database

Refrence model: https://doi.org/10.1007/s10044-021-00970-4

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.layers import *
import cv2 
import os
import pathlib
import PIL
import random
import pickle


print('Libraries are imported ')

Libraries are imported 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exploring data

In [ ]:
print('Loading data about COVID-19 images')
covid_csv_data = pd.read_csv("/content/drive/MyDrive/Dataset-Copy/metadata/COVID.metadata.csv")
covid_csv_data.head()

Loading data about COVID-19 images


,FILE NAME,FORMAT,SIZE,URL
0,COVID 1,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
1,COVID 2,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
2,COVID 3,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
3,COVID 4,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
4,COVID 5,PNG,256*256,https://sirm.org/category/senza-categoria/covi...


## Loading image data from dataset


In [ ]:
DIRECTORY = '/content/drive/MyDrive/Dataset-Copy/dataset'
CATEGORIES = os.listdir(DIRECTORY)
print(CATEGORIES)

['NORMAL', 'Viral Pneumonia', 'COVID']


In [ ]:
IMG_SIZE = 224
data = []

for category in CATEGORIES:
  folder = os.path.join(DIRECTORY, category)
  label = CATEGORIES.index(category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder, img)
    
    try:
      img_arr = cv2.imread(img_path)
      img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
      data.append([img_arr, label])  
      
      
    except:
      pass
    

In [ ]:
data[1]

[array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [35, 35, 35],
         [30, 30, 30],
         [30, 30, 30]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [46, 46, 46],
         [36, 36, 36],
         [36, 36, 36]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [43, 43, 43],
         [44, 44, 44],
         [42, 42, 42]],
 
        ...,
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]]], dtype=uint8), 0]

In [ ]:
len(data)

1500

In [ ]:
random.shuffle(data)

In [ ]:
X = []
Y = []

for features, labels in data:
  X.append(features)
  Y.append(labels)

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
pickle.dump(X, open('X1.pkl', 'wb'))
pickle.dump(Y, open('Y1.pkl', 'wb'))

In [ ]:
#X = pickle.load(open('/content/drive/MyDrive/X1.pkl', 'rb'))
#Y = pickle.load(open('/content/drive/MyDrive/Y1.pkl', 'rb'))

In [ ]:
X = X/255

In [ ]:
np.array(X[1])

array([[[0.04313725, 0.04313725, 0.04313725],
        [0.07058824, 0.07058824, 0.07058824],
        [0.11764706, 0.11764706, 0.11764706],
        ...,
        [0.56470588, 0.56470588, 0.56470588],
        [0.70196078, 0.70196078, 0.70196078],
        [0.91372549, 0.91372549, 0.91372549]],

       [[0.02352941, 0.02352941, 0.02352941],
        [0.0627451 , 0.0627451 , 0.0627451 ],
        [0.09411765, 0.09411765, 0.09411765],
        ...,
        [0.54117647, 0.54117647, 0.54117647],
        [0.69803922, 0.69803922, 0.69803922],
        [0.92156863, 0.92156863, 0.92156863]],

       [[0.01568627, 0.01568627, 0.01568627],
        [0.05098039, 0.05098039, 0.05098039],
        [0.07843137, 0.07843137, 0.07843137],
        ...,
        [0.5254902 , 0.5254902 , 0.5254902 ],
        [0.70980392, 0.70980392, 0.70980392],
        [0.90980392, 0.90980392, 0.90980392]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

### Splitting the data

In [ ]:
# split training and test data

(X_train, x_test, Y_train, y_test) = train_test_split(X, Y,
	test_size=0.20, stratify=Y)



In [ ]:
print(len(X_train))
print(len(x_test))

1200
300


In [ ]:
from tensorflow.keras.applications import VGG19, ResNet50V2, DenseNet201, InceptionResNetV2, InceptionV3, Xception, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

def train_model(path, train_images=None, train_labels = None, 
                test_images = None, test_labels = None, 
                model_name = None, epochs =80, learning_rate = 0.0001,
                input_shape = (224,224,3), classes=3, batch_size = 16, 
                classifier_activation='softmax',
                callbacks = None):
    

    base_model = None
    if model_name == 'vgg19':        
        base_model = VGG19(weights = None, include_top = False, input_shape = input_shape)
     
    if model_name == 'resnet50_v2':
        base_model = ResNet50V2(weights = None, include_top = False, input_shape = input_shape)
             
    if model_name == 'inception_resnet_v2':        
        base_model = InceptionResNetV2(weights = None, include_top = False, input_shape = input_shape)
    
    if model_name == 'densenet201':        
        base_model = DenseNet201(weights = None, include_top = False, input_shape = input_shape)
          
    if model_name == 'inception_v3':        
        base_model = InceptionV3(weights = None, include_top = False, input_shape = input_shape)
        
    if model_name == 'xception':        
        base_model = Xception(weights = None, include_top = False, input_shape = input_shape)
        
    if model_name == 'mobilenet_v2':        
        base_model = MobileNetV2(weights = None, include_top = False, input_shape = input_shape)

    x = base_model.output         
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Dense(classes, activation=classifier_activation)(x)

    model = tf.keras.Model(inputs = base_model.input, outputs = output)

    optimizer = Adam(learning_rate = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

    model.compile(optimizer = optimizer,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
        
    results = model.fit(train_images, train_labels, epochs = epochs,
                        validation_data = (test_images, test_labels), 
                        batch_size=batch_size, 
                        callbacks = callbacks
                        )
    
    
    
    save_model = path + model_name + '.h5'
    model.save(save_model)
    
    return results    


In [ ]:
IMG_SIZE = 224

print('\nTraining Inception v3\n')
inception = train_model(DIRECTORY, X_train, Y_train,
                        x_test, y_test, model_name = "inception_v3",
                        epochs = 30, input_shape = (IMG_SIZE,IMG_SIZE,3),
                        classes = len(CATEGORIES))

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, 
                           mode='min', restore_best_weights=True)
callbacks = [early_stop]

print('\nTraining DenseNet 201\n')
densenet = train_model(DIRECTORY, X_train, Y_train,
                       x_test, y_test, model_name="densenet201",
                       epochs=30, input_shape = (IMG_SIZE,IMG_SIZE,3),
                       classes = len(CATEGORIES),
                       callbacks = callbacks)

print('\nTraining ResNet50 v2\n')
resnet = train_model(DIRECTORY, X_train, Y_train,
                     x_test, y_test, model_name="resnet50_v2",
                     epochs=30, input_shape = (IMG_SIZE,IMG_SIZE,3),
                     classes = len(CATEGORIES),
                     callbacks = callbacks)


Training Inception v3

Epoch 1/30
75/75 [==============================] - 68s 377ms/step - loss: 0.5654 - accuracy: 0.7553 - val_loss: 1.7166 - val_accuracy: 0.3333
Epoch 2/30
75/75 [==============================] - 25s 340ms/step - loss: 0.2568 - accuracy: 0.9101 - val_loss: 1.8738 - val_accuracy: 0.3333
Epoch 3/30
75/75 [==============================] - 26s 342ms/step - loss: 0.2632 - accuracy: 0.9079 - val_loss: 4.4253 - val_accuracy: 0.3333
Epoch 4/30
75/75 [==============================] - 26s 342ms/step - loss: 0.1998 - accuracy: 0.9270 - val_loss: 4.2487 - val_accuracy: 0.3700
Epoch 5/30
75/75 [==============================] - 26s 342ms/step - loss: 0.1750 - accuracy: 0.9428 - val_loss: 9.1795 - val_accuracy: 0.3333
Epoch 6/30
75/75 [==============================] - 26s 342ms/step - loss: 0.1182 - accuracy: 0.9616 - val_loss: 3.9763 - val_accuracy: 0.6300
Epoch 7/30
75/75 [==============================] - 26s 343ms/step - loss: 0.0670 - accuracy: 0.9781 - val_loss: 3.061

In [ ]:
def ensemble(x, weights, models): 
    '''
    returns a weighted average of predictions made by the models\n
    x -> input image \n
    weights -> a list of weights \n
    models -> a list of models\n    
    '''      
    outputs = []    
    for model in models:                
        outputs.append(list(model.predict(x)[0]))                
    
    outputs = np.array(outputs)
    avg = np.average(a=outputs,axis=0,weights=weights)
    return avg


def equal(pred, label):
  pred_id = np.argmax(pred)
  if (pred_id == label):
    return True
  else:
    return False


def accuracy(predicted_values, y_truths):
  '''
  returns accuracy\n
  predicted_values = a numpy array containing the predictions\n
  y_truths = a numpy array containing the truth values\n  
  '''
  total = len(y_truths)
  correct = 0
  for i in range(len(y_truths)):
    if equal(predicted_values[i],y_truths[i]):
      correct += 1
  acc = correct / total
  return acc


def generate_weights(x_val, y_val, models):
    '''
    returns a list of weights
    '''
    accuracy = []
    weights = np.full((1,len(models)), 100.0)
    for model in models:
      acc = model.evaluate(x_val, y_val)[1]
      accuracy.append(100*acc)

    weights = weights - accuracy
    weights = weights**2

    sum = np.sum(weights)
    weights = weights/sum
    weights = 1/weights

    weights = weights**2
    sum = np.sum(weights)

    weights = weights/sum
    
    return weights

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix



inception_path = '/content/drive/MyDrive/Dataset-Copy/datasetinception_v3.h5'
resnet_path = '/content/drive/MyDrive/Dataset-Copy/datasetresnet50_v2.h5'
densenet_path = '/content/drive/MyDrive/Dataset-Copy/datasetdensenet201.h5'




inception_model = tf.keras.models.load_model(inception_path)
resnet_model = tf.keras.models.load_model(resnet_path)
densenet_model = tf.keras.models.load_model(densenet_path)

models = [densenet_model,resnet_model,inception_model]

w = generate_weights(x_test,y_test,models)[0] #generating weights
print("Weights: ", w)

predictions = []
for i in range(len(x_test)):
  pred = ensemble(x_test[i].reshape(-1,IMG_SIZE,IMG_SIZE,3),w,models)
  predictions.append(pred)

print("Accuracy: ",accuracy(predictions,y_test))

y_pred = np.argmax(np.array(predictions), axis=1)

print("The classification report: ")
print(classification_report(y_pred=y_pred, y_true=y_test))
print()
print("Confusion Matrix: ")
print(confusion_matrix(y_pred=y_pred, y_true=y_test))

10/10 [==============================] - 5s 152ms/step - loss: 0.1227 - accuracy: 0.9667
Weights:  [0.09410432 0.11992928 0.7859664 ]
Accuracy:  0.9666666666666667
The classification report: 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       100
           1       0.98      0.92      0.95       100
           2       0.98      0.98      0.98       100

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300


Confusion Matrix: 
[[100   0   0]
 [  6  92   2]
 [  0   2  98]]


# ================RANDOM FOREST======================

In [ ]:
activation = 'sigmoid'

feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (IMG_SIZE, IMG_SIZE, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

In [ ]:
#Add layers for deep learning prediction
x1 = feature_extractor.output  
x1 = Dense(128, activation = activation, kernel_initializer = 'he_uniform')(x1)
prediction_layer = Dense(3, activation = 'softmax')(x1)             #3 is number of class 

# Make a new model combining both feature extractor and x
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94_input (InputLayer) [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization_94 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 224, 224, 32)      9248      
_________________________________________________________________
batch_normalization_95 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 112, 112, 64)      1849

In [ ]:
X_for_RF = feature_extractor.predict(X_train) #This is out X input to RF

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 70, random_state = 42)

In [ ]:

# Train the model on training data
RF_model.fit(X_for_RF, Y_train) #For sklearn no one hot encoding

#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)
#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_feature)


In [ ]:

from sklearn import metrics

print("The classification report: ")
print(classification_report(y_pred=y_pred, y_true=y_test))

print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_RF))

#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(y_test, prediction_RF)
#print(cm)
cm

The classification report: 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       100
           1       0.98      0.92      0.95       100
           2       0.98      0.98      0.98       100

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300

Accuracy =  0.9133333333333333


array([[97,  2,  1],
       [ 6, 88,  6],
       [ 0, 11, 89]])